In [1]:
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
from N2_dataprep_time import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

In [11]:
epochs = 11
frame_in = 24
frame_out = 1
test_ratio = 0.2

In [16]:
class CNNLSTM(Model):
    def __init__(self):
        super().__init__()

    def setup(self, frame_in, frame_out, filters_=24, kernel_size_=5, pool_size_=4):
        self.model = Sequential()
        self.model.add(TimeDistributed(Conv1D(filters=filters_, kernel_size=kernel_size_, activation='relu'), input_shape=(None, frame_in, 1)))
        self.model.add(TimeDistributed(MaxPooling1D(pool_size=pool_size_)))
        self.model.add(TimeDistributed(Flatten()))
        self.model.add(LSTM(24, activation='relu'))
        self.model.add(Dense(frame_out))
        self.model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [17]:
str_fn = '-'+str(frame_in)+'-'+str(frame_out)
try:
    train_inp = pk.load(open("temp/trinp"+str_fn, 'rb'))
    train_oup = pk.load(open("temp/troup"+str_fn, 'rb'))
    test_inp = pk.load(open("temp/teinp"+str_fn, 'rb'))
    test_oup = pk.load(open("temp/teoup"+str_fn, 'rb'))
except:
    train_inp, train_oup, test_inp, test_oup = compound(setup(test_ratio))
    pk.dump(train_inp, open("temp/trinp"+str_fn, 'wb'))
    pk.dump(train_oup, open("temp/troup"+str_fn, 'wb'))
    pk.dump(test_inp, open("temp/teinp"+str_fn, 'wb'))
    pk.dump(test_oup, open("temp/teoup"+str_fn, 'wb'))

In [18]:
train_limit = len(train_inp)
test_limit = len(test_inp)

c = CNNLSTM()
c.setup(frame_in, frame_out)
try:
    c.setData(train_inp[:int(train_limit)], train_oup[:int(train_limit)])
except:
    c.setData(train_inp, train_oup)
# c.summarize()

In [19]:
c.train(epochs_=epochs, verbose_=2, test=(test_inp, test_oup))

Epoch 1/11


ValueError: in user code:

    C:\Users\clementich\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\clementich\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\clementich\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\clementich\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\clementich\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        r

In [ ]:
p = Prediction(test_inp, test_oup, c, frame_in, frame_out)